This notebook allow you to print results of models in tables

In [1]:
import os ; os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [2]:
from systemtools.hayj import *
from systemtools.location import *
from systemtools.basics import *
from systemtools.file import *
from systemtools.printer import *
from twinews.utils import *
from twinews.evaluation.utils import *

In [3]:
df = printReport\
(
    model='lda',
    splitVersion=2,
    noSubsampling=True,
    metaFilter=\
    {
        # "implementation": "gensim-lda",
    },
    metricsFilter=None, # {'ndcg', 'ndcg@10'},
    sortBy='ndcg',
)

These values are common to all rows:

	- useExtraNews: False
	- lowercase: True
	- maxUsers: None
	- splitVersion: 2
	- maxDF: 300
	- model: lda
	- ldaLearningMethod: online
	- ldaLearningDecay: 0.7


,id,distance,doLemmatization,historyRef,implementation,ldaLearningOffset,maxDocuments,maxIter,minDF,nbTopics,useTFIDF,map,mrr,ndcg,ndcg@10,ndcg@100,p@10,p@100
2,lda-d4f1d,cosine,False,30,gensim-lda,1.0,300000,60,0.0005,100,False,0.160605,0.334149,0.418971,0.198836,0.330036,0.128878,0.043620
14,lda-b123f,cosine,False,30,gensim-lda,1.0,100000,120,0.0005,100,False,0.156168,0.325272,0.414157,0.192609,0.322127,0.124219,0.042599
1,lda-d1220,cosine,False,0.3,gensim-lda,1.0,100000,60,0.0005,100,False,0.131120,0.274326,0.386483,0.157042,0.287863,0.104363,0.040735
15,lda-af807,cosine,False,30,gensim-lda,1.0,10000,60,0.0005,100,False,0.124114,0.280166,0.379418,0.157717,0.268787,0.102465,0.036118
9,lda-759de,cosine,False,0.6,gensim-lda,1.0,10000,60,0.0005,100,False,0.119143,0.273456,0.374841,0.151466,0.264193,0.099220,0.036128
12,lda-64d0c,cosine,False,1.0,gensim-lda,1.0,10000,60,0.0005,100,False,0.120190,0.274310,0.374784,0.152779,0.260727,0.099453,0.035244
13,lda-2fa8e,euclidean,False,0.3,gensim-lda,1.0,10000,60,0.0005,100,False,0.108196,0.254482,0.361689,0.137830,0.245750,0.091047,0.034073
7,lda-b849e,cosine,False,0.3,gensim-lda,1.0,10000,60,0.0005,300,False,0.105073,0.244526,0.356511,0.132101,0.236323,0.086011,0.033017
17,lda-7786b,cosine,True,0.3,gensim-lda,1.0,10000,60,0.0005,100,False,0.103803,0.235455,0.355055,0.127154,0.236139,0.084533,0.033966
8,lda-d3b82,cosine,False,0.3,gensim-lda,1.0,10000,60,0.0005,100,False,0.101780,0.230012,0.353350,0.125064,0.233837,0.083030,0.033504


In [4]:
df = printReport\
(
    model='nmf',
    splitVersion=2,
    noSubsampling=True,
    metaFilter=\
    {
        # "implementation": "gensim-lda",
    },
    metricsFilter=None, # {'ndcg', 'ndcg@10'},
    sortBy='ndcg',
)

These values are common to all rows:

	- useExtraNews: False
	- lowercase: True
	- maxUsers: None
	- splitVersion: 2
	- nmfAlpha: 0.1
	- maxDF: 300
	- model: nmf
	- implementation: sklearn-nmf
	- minDF: 0.0005
	- distance: cosine


,id,doLemmatization,historyRef,maxDocuments,maxIter,nbTopics,nmfInit,nmfL1Ratio,map,mrr,ndcg,ndcg@10,ndcg@100,p@10,p@100
8,nmf-9cd4f,False,30,100000,200,100,nndsvd,0.0,0.239128,0.455548,0.493229,0.296854,0.421229,0.186212,0.049218
0,nmf-330c8,False,30,10000,200,100,nndsvd,0.0,0.234911,0.446258,0.489016,0.291090,0.415755,0.182861,0.048848
7,nmf-b1354,False,30,100000,200,100,None,0.0,0.233865,0.448569,0.488428,0.290335,0.415213,0.181610,0.048770
2,nmf-ad87b,False,30,100000,400,100,nndsvd,0.0,0.232042,0.444272,0.486531,0.287608,0.412681,0.180717,0.048566
3,nmf-84f4b,False,0.6,100000,200,100,nndsvd,0.0,0.228404,0.436960,0.483361,0.283407,0.410103,0.179755,0.048814
4,nmf-d9bba,False,0.3,100000,200,100,nndsvd,0.0,0.213807,0.406514,0.468633,0.263998,0.393407,0.171053,0.048350
9,nmf-ef428,False,30,100000,200,100,nndsvd,0.5,0.212487,0.417334,0.467817,0.264327,0.387968,0.166306,0.046467
1,nmf-b61b9,True,0.5,1000,100,30,nndsvd,0.0,0.150380,0.318810,0.409265,0.186974,0.317141,0.122490,0.042544
6,nmf-09f9c,True,0.3,1000,200,30,nndsvd,0.0,0.149538,0.311292,0.407733,0.184111,0.316140,0.121553,0.042904
5,nmf-dea50,True,0.3,10000,200,30,nndsvd,0.0,0.145358,0.304368,0.403756,0.177553,0.312424,0.117642,0.043049


In [11]:
def highlight_col(x):
    r = 'background-color: rgba(150, 150, 150, 0.3)'
    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
    df1.iloc[:, 1] = r
    return df1    
df.style.apply(highlight_col, axis=None)

,id,doLemmatization,historyRef,maxDocuments,maxIter,nbTopics,nmfInit,nmfL1Ratio,map,mrr,ndcg,ndcg@10,ndcg@100,p@10,p@100
8,nmf-9cd4f,False,30,100000,200,100,nndsvd,0,0.239128,0.455548,0.493229,0.296854,0.421229,0.186212,0.0492185
0,nmf-330c8,False,30,10000,200,100,nndsvd,0,0.234911,0.446258,0.489016,0.29109,0.415755,0.182861,0.0488482
7,nmf-b1354,False,30,100000,200,100,None,0,0.233865,0.448569,0.488428,0.290335,0.415213,0.18161,0.0487696
2,nmf-ad87b,False,30,100000,400,100,nndsvd,0,0.232042,0.444272,0.486531,0.287608,0.412681,0.180717,0.0485665
3,nmf-84f4b,False,0.6,100000,200,100,nndsvd,0,0.228404,0.43696,0.483361,0.283407,0.410103,0.179755,0.0488142
4,nmf-d9bba,False,0.3,100000,200,100,nndsvd,0,0.213807,0.406514,0.468633,0.263998,0.393407,0.171053,0.0483502
9,nmf-ef428,False,30,100000,200,100,nndsvd,0.5,0.212487,0.417334,0.467817,0.264327,0.387968,0.166306,0.0464671
1,nmf-b61b9,True,0.5,1000,100,30,nndsvd,0,0.15038,0.31881,0.409265,0.186974,0.317141,0.12249,0.0425439
6,nmf-09f9c,True,0.3,1000,200,30,nndsvd,0,0.149538,0.311292,0.407733,0.184111,0.31614,0.121553,0.0429041
5,nmf-dea50,True,0.3,10000,200,30,nndsvd,0,0.145358,0.304368,0.403756,0.177553,0.312424,0.117642,0.0430487
